In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [2]:
path_train = "UJIndoorLoc/trainingData.csv"
path_validation = "UJIndoorLoc/validationData.csv"

In [3]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.ix[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths))

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  after removing the cwd from sys.path.
c:\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [5]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [6]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.ix[:,0:520]))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [7]:
nb_epochs = 20
batch_size = 10
input_size = 520
num_classes = 13

In [8]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model

In [13]:
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
    

In [14]:
e = encoder()

c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="tanh", input_dim=520, use_bias=True)`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python\python35\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", use_bias=True)`
  after removing the cwd from sys.path.
c:\python\python35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="tanh", use_bias=True)`
  """


In [15]:
d = decoder(e)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", input_dim=64, use_bias=True)`
  
c:\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="tanh", use_bias=True)`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python\python35\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(520, activation="tanh", use_bias=True)`
  after removing the cwd from sys.path.


In [16]:
d.fit(train_X, train_X, epochs=nb_epochs, batch_size=batch_size)

Epoch 1/20
13912/13912 [==============================] - 6s - loss: 0.7243     
Epoch 2/20
13912/13912 [==============================] - 6s - loss: 0.6997     
Epoch 3/20
13912/13912 [==============================] - 6s - loss: 0.6939     
Epoch 4/20
13912/13912 [==============================] - 6s - loss: 0.6912     
Epoch 5/20
13912/13912 [==============================] - 6s - loss: 0.6890     
Epoch 6/20
13912/13912 [==============================] - 6s - loss: 0.6870     
Epoch 7/20
13912/13912 [==============================] - 6s - loss: 0.6854     
Epoch 8/20
13912/13912 [==============================] - 6s - loss: 0.6838     
Epoch 9/20
13912/13912 [==============================] - 6s - loss: 0.6825     
Epoch 10/20
13912/13912 [==============================] - 6s - loss: 0.6813     
Epoch 11/20
13912/13912 [==============================] - 6s - loss: 0.6804     
Epoch 12/20
13912/13912 [==============================] - 6s - loss: 0.6794     
Epoch 13/20
13912/13912 [

In [17]:
def classifier(d):
    d.add(Dense(64, input_dim=64, activation='relu', bias=True))
    #num_to_remove = 3
    #for i in range(num_to_remove):
        #d.pop()
    d.add(Dropout(0.75))
    d.add(Dense(32, activation='relu', bias=True))
    d.add(Dense(16, activation='relu', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d

In [18]:
c = classifier(d)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", input_dim=64, use_bias=True)`
  
c:\python\python35\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", use_bias=True)`
  import sys
c:\python\python35\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", use_bias=True)`
  
c:\python\python35\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(13, activation="softmax", use_bias=True)`
  if __name__ == '__main__':


In [19]:
c.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=nb_epochs, batch_size=batch_size)

Train on 13912 samples, validate on 6018 samples
Epoch 1/20
13912/13912 [==============================] - 7s - loss: 1.0121 - acc: 0.6251 - val_loss: 0.4167 - val_acc: 0.8538
Epoch 2/20
13912/13912 [==============================] - 7s - loss: 0.5237 - acc: 0.8052 - val_loss: 0.3105 - val_acc: 0.8865
Epoch 3/20
13912/13912 [==============================] - 7s - loss: 0.4558 - acc: 0.8303 - val_loss: 0.3111 - val_acc: 0.8922
Epoch 4/20
13912/13912 [==============================] - 7s - loss: 0.4208 - acc: 0.8452 - val_loss: 0.3062 - val_acc: 0.8956
Epoch 5/20
13912/13912 [==============================] - 7s - loss: 0.3744 - acc: 0.8562 - val_loss: 0.2947 - val_acc: 0.8868
Epoch 6/20
13912/13912 [==============================] - 8s - loss: 0.3427 - acc: 0.8684 - val_loss: 0.2908 - val_acc: 0.8830
Epoch 7/20
13912/13912 [==============================] - 8s - loss: 0.3515 - acc: 0.8687 - val_loss: 0.2347 - val_acc: 0.9043
Epoch 8/20
13912/13912 [==============================] - 8s -

In [20]:
loss, acc = c.evaluate(test_AP_features, test_labels)

 800/1111 [====================>.........] - ETA: 0s

In [21]:
print (loss, acc)

1.043847667 0.745274527453


### Notes:

- This program has a little bit higher accuracy, around 3%, compared with the other sample. I don't think it has a larger 